In [58]:
import osmnx as ox
import matplotlib as plt
import geopandas as gpd
from k_means_constrained import KMeansConstrained
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization
from tensorflow.keras.initializers import he_normal
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model



In [2]:
# load in Data
Porto_data = pd.read_csv(r'C:\Users\NimaMehrafar\UC\UC_Project\porto_data_2')
SanFrancisco_data = pd.read_csv(r'C:\Users\NimaMehrafar\UC\UC_Project\san_francisco_data_2')
Tokyo_data = pd.read_csv(r'C:\Users\NimaMehrafar\UC\UC_Project\tokyo_data_2')

In [3]:
clf = KMeansConstrained(
     n_clusters=85,
    size_min=40,
    size_max=60, 
    random_state=0)

In [4]:
#Preprocessing
Porto_data = Porto_data[['element_type','highway','n_type']]
label_encoder = LabelEncoder()
Porto_data['highway'] = label_encoder.fit_transform(Porto_data['highway'])
Porto_data['element_type'] = label_encoder.fit_transform(Porto_data['element_type'])


In [5]:
#Clustering
group_size = 30
clf.fit_predict(Porto_data.values)
clusters = clf.labels_
Porto_data['cluster'] = clusters

In [6]:
# X and Y
X = Porto_data[['element_type','highway']]
y = Porto_data['n_type']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [50]:
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)
unique_labels = set(y_train)
print("Unique Labels:", unique_labels)
y_train = y_train - 1
y_test = y_test - 1


Unique Labels: {1, 2, 3, 4, 5}


In [51]:
model = Sequential()
model.add(Dense(64, input_dim=2, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(5, activation='softmax'))  # Use 'sigmoid' for binary classification
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/10
83/83 [==============================] - 3s 12ms/step - loss: 1.3992 - accuracy: 0.4375 - val_loss: 1.1601 - val_accuracy: 0.7221
Epoch 2/10
83/83 [==============================] - 1s 10ms/step - loss: 1.0544 - accuracy: 0.6222 - val_loss: 0.9316 - val_accuracy: 0.7221
Epoch 3/10
83/83 [==============================] - 1s 9ms/step - loss: 0.8681 - accuracy: 0.7083 - val_loss: 0.8913 - val_accuracy: 0.7221
Epoch 4/10
83/83 [==============================] - 1s 8ms/step - loss: 0.7994 - accuracy: 0.7148 - val_loss: 0.8635 - val_accuracy: 0.7221
Epoch 5/10
83/83 [==============================] - 1s 8ms/step - loss: 0.7789 - accuracy: 0.7144 - val_loss: 0.8694 - val_accuracy: 0.7221
Epoch 6/10
83/83 [==============================] - 1s 8ms/step - loss: 0.7688 - accuracy: 0.7163 - val_loss: 0.8012 - val_accuracy: 0.7221
Epoch 7/10
83/83 [==============================] - 1s 13ms/step - loss: 0.7668 - accuracy: 0.7167 - val_loss: 0.7633 - val_accuracy: 0.7221
Epoch 8/10
83/83 

In [56]:
y_pred = model.predict(X_test)
_, accuracy = model.evaluate(X_test, y_test)
accuracy

26/26 [==============================] - 0s 5ms/step - loss: 0.6851 - accuracy: 0.7742


0.7741546034812927

In [59]:
embedding_model = Model(inputs=model.input, outputs=model.layers[-2].output)


In [67]:
embeddings = embedding_model.predict(X_train)

104/104 [==============================] - 1s 5ms/step


In [69]:
len(embeddings) #

3309